<a href="https://colab.research.google.com/github/ProfDoeg/Colegio_Invisible/blob/main/03_cuaderno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/ProfDoeg/Colegio_Invisible.git
! pip install cryptos
! pip install eciespy


In [ ]:
 #! rm -r Colegio_Invisible

# Basic Cryptography and Script Writing

## Outline

In this deeper dive we cover:

- asymmetric cryptography
- advanced scripts

By the end of the exposition you will:

- understand the features of **asymmetric cryptography**
- be able to generate a new **private key** object
- be able to access the byte or hex value of the private key
- be able to import a previously generated **private key** from bytes
- be able to extract a **public key** from the private key
- be able to access the byte or hex value of the public key
- understand ECDSA digital signature
- be able to **sign** a message with a private key
- be able to **verify** a message signature with a public key
- understand ECIES
- be able to **ECIES encrypt** with a public key
- be able to **ECIES decrypt** with a private key

# Asymmetric Cryptography

Asymmetric cryptography, or public-key cryptography, is a system that uses pairs of keys: public keys (which may be known to others), and private keys (which may never be known by any except the owner).

- Public keys are generated from private keys
- Public keys are shown to others
- Private keys are held private

<div>
<img src="attachment:asymmetric.png" width="500"/>
</div>

**ENCRYPTION**
For encryption and decryption:
- the recipients public key is used to encrypt
- the recipients private key is used to decrypt

**DIGITAL SIGNATURE**
For signature and verification
- the sender's private key is used the sign the hash of a text to generate a signature
- the sender's public key is used to turn the signature back into the hash and compared to the hash of the text. if the hashes match then the text/signature/pubkey fit and the signature is deemed valid.

**ECC** and RSA are different kinds of asymmetric cryptography.
Elliptic-curve cryptography (ECC) is an approach to public-key cryptography based on the algebraic structure of elliptic curves over finite fields. ECC allows smaller keys compared to RSA.

**secp256k1** refers to the parameters of the elliptic curve used in Ethereum's, Bitcoin's, and their derivatives' public-key cryptography.

In [ ]:
import ecies
import eth_keys

In [ ]:
! ls Colegio_Invisible/scripts

## Generate Private Key

In [ ]:
#this generates a private key object
privKey1 = ecies.utils.generate_eth_key()
privKey1, type(privKey1)

In [ ]:
#this generates a private key
privKey2 = ecies.utils.generate_eth_key()
privKey2, type(privKey2)

## Dump Private Key Bytes

In [ ]:
#here we can dump the contents of the private key as bytes
privKey1_bytes=privKey1.to_bytes()
privKey1_bytes,privKey1_bytes.hex()

## Import Previously Generated Private Key From Bytes

In [ ]:
#here we are generating a key object from previously generated bytes
privKey1_copy=eth_keys.keys.PrivateKey(privKey1_bytes)
privKey1_copy,type(privKey1_copy)

## Generate Private Key and Save in AES Password Protected File Using a Python Script

In [ ]:
! ls Colegio_Invisible/scripts
! cat Colegio_Invisible/scripts/ecc_generate.py

In [ ]:
! mkdir keys

In [ ]:
! python Colegio_Invisible/scripts/ecc_generate.py keys/my_privkey.enc

# Derive Public Key From Private Key

In [ ]:
pubKey1=privKey1.public_key
pubKey1

In [ ]:
pubKey2=privKey2.public_key
pubKey2

## Dump Public Key Bytes

In [ ]:
pubKey1_bytes=pubKey1.to_bytes()
pubKey1_bytes

## Import Previously Generated Private Key From Bytes

In [ ]:
pubKey1_import=eth_keys.keys.PublicKey(pubKey1_bytes)
pubKey1_import,type(pubKey1_import)

# Extracting Public Key and Saving to AES Password Protected File Using Python Script

In [ ]:
! cat Colegio_Invisible/scripts/ecc_pubkey_extract.py

In [ ]:
! python Colegio_Invisible/scripts/ecc_pubkey_extract.py keys/my_privkey.enc keys/my_pubkey.enc

## Dumping the Contents of an AES Password Protected Key File Using a Python Script

In [ ]:
! cat Colegio_Invisible/scripts/ecc_keydump.py 

In [ ]:
! python Colegio_Invisible/scripts/ecc_keydump.py keys/my_pubkey.enc

In [ ]:
pubkey=!{'python Colegio_Invisible/scripts/ecc_keydump.py keys/my_pubkey.enc'}
pubkey

In [ ]:
bytes.fromhex(pubkey[-1])

---
## Digital Signature


Digital signature is a means by which we ensure the authenticity and integrity of a message.

- authenticity relates to the source of a message
- integrity relates to the fact that a message has not been tampered with by adding, removing or changing its content

**ECDSA** (Elliptic Curve Digital Signature Algoritm) is an algorithm for performing digital signature using elliptic curve cryptography. 

- signature is performed using a private key and a text in byte form
- the signature algortitm is used generate a signature 
- verification is performed using the triplet signature/text/publickey
 
https://en.wikipedia.org/wiki/Elliptic_Curve_Digital_Signature_Algorithm

*SECURITY NOTE:* the text and signature can be used to recover the public key value

<br></br>
<div>
<img src="attachment:Digital-signature-signing-and-verification.png" width="500"/>
</div>


## Sign Message

In [ ]:
### lets make two messages
message_A=b'this is message A'
message_B=b'this is message B'

In [ ]:
### now lets criss-cross the two keys and two messages
signature_1A=privKey1.sign_msg(message_A)
signature_1B=privKey1.sign_msg(message_B)
signature_2A=privKey2.sign_msg(message_A)
signature_2B=privKey2.sign_msg(message_B)

## Dump the Contents of the Signature as Bytes

In [ ]:
sig_bytes=signature_1A.to_bytes()
sig_bytes

In [ ]:
new_sig=eth_keys.datatypes.Signature(sig_bytes)
new_sig, type(new_sig)

## Sign File with Private Key using Python Script

In [ ]:
! cat Colegio_Invisible/scripts/ecc_sign.py

In [ ]:
! python Colegio_Invisible/scripts/ecc_sign.py  keys/my_privkey.enc '' Colegio_Invisible/img/colegio_invisible.jpeg ic_img.sig

In [ ]:
! python Colegio_Invisible/scripts/sha_256.py Colegio_Invisible/img/colegio_invisible.jpeg

## Verify Signature

In [ ]:
#if signature, message and key match then True
signature_1A.verify_msg(message_A,pubKey1)

In [ ]:
#if message does not match
signature_1A.verify_msg(message_B,pubKey1)

In [ ]:
#if public key does not match
signature_1A.verify_msg(message_A,pubKey2)

In [ ]:
signature_2B.verify_msg(message_B,pubKey2)

## Verify Signature/Message/PublicKey Triplet with Python Script

In [ ]:
! cat Colegio_Invisible/scripts/ecc_verify.py

In [ ]:
! python Colegio_Invisible/scripts/ecc_verify.py keys/my_pubkey.enc '' Colegio_Invisible/img/colegio_invisible.jpeg ic_img.sig

# ECIES

ECIES stands for Elliptic Curve Integrated Encryption Scheme. It is a hybrid scheme where we use both ECC and AES.

The process goes as follows:

***SENDING...***

1. The sender generates a one-time-use AES encryption key (session key)
2. The sender encrypts the plain text with the **AES session key** and produces a cipher text
3. The sender then encrypts the AES session key with the **reciever public key**
4. The sender then packages the encrypted file and the encrypted key together

***RECIEVING...***

5. The **reciever private key** is used to decrypt the **encrypted AES session key**
6. The **decrypted AES session key** is used to decrypt the ciper text...recovering the plain text

---
---

## Encrypt

In [ ]:
ciphertext_1A=ecies.encrypt(pubKey1.to_hex(),msg=message_A)
ciphertext_1A

In [ ]:
ciphertext_2B=ecies.encrypt(pubKey2.to_hex(),msg=message_B)
ciphertext_2B

## Encrypt File with Python Script

In [ ]:
! cat Colegio_Invisible/scripts/ecc_encrypt.py

In [ ]:
! python Colegio_Invisible/scripts/ecc_encrypt.py keys/my_pubkey.enc '' Colegio_Invisible/img/colegio_invisible.jpeg colegio_invisible.enc

## Decrypt

In [ ]:
ecies.decrypt(privKey1.to_hex(),ciphertext_1A)

In [ ]:
ecies.decrypt(privKey2.to_hex(),ciphertext_2B)

In [ ]:
#if we use the wrong key...
ecies.decrypt(privKey1.to_hex(),ciphertext_2B)

## Decrypt Using Python Script

In [ ]:
! python Colegio_Invisible/scripts/ecc_decrypt.py keys/my_privkey.enc '' colegio_invisible.enc colegio_invisible_dec.jpeg

In [ ]:
from IPython.display import Image
display(Image('colegio_invisible_dec.jpeg'))